In [39]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from datetime import date
from pathlib import Path
import pickle
import codebase
from codebase import ml_pipeline

topo_vars: gauge_id;gauge_lat;gauge_lon;elev_mean;slope_mean;area_gages2;

| topo_vars    | GRDC eq.  | Notes      |
|--------------|-----------|------------|
| gauge_id     | grdc_no   |            |
| gauge_lat    | lat_pp    | from Google Maps inspection, more accurate than lat_org   |
| gauge_lon    | long_pp   | from Google Maps inspection, more accurate than long_org  |
| elev_mean    |           |            |
| slope_mean   |           |            |
| area_gages2  | area_calc | some 'area' columns have -999, so using 'area_calc'       |

All subregion files are the same.

In [28]:
## define experimental set-up

# grdc stored as floats in the downlaod jsons
grdc_id = 1834101
basin_str = 'magdalena'

In [3]:
## Other variables and filepaths
grdc_dir = "/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/"
met_dir = "/global/scratch/users/ann_scheliga/era5_data/"
res_dir = "/global/scratch/users/ann_scheliga/CYGNSS_daily/time_series/"
basin_data_dir = "/global/scratch/users/ann_scheliga/basin_forcing_processed/"

In [29]:
subregions_meta = gpd.read_file(grdc_dir + basin_str + '_subregions.geojson')
stations_meta = gpd.read_file(grdc_dir + basin_str + '_stationbasins.geojson')

In [8]:
test_subregion = gpd.read_file(grdc_dir + 'colorado' + '_subregions.geojson')

In [30]:
print(test_subregion.shape, subregions_meta.shape)

(841, 14) (841, 14)


In [31]:
(test_subregion == subregions_meta).sum()

GmlID         841
FID           841
OBJECTID      841
WMOBB         841
SUBREGNUM     841
SUBREGNAME    841
SUBREG_DES    841
SUM_SUB_AR    841
REGNUM        841
REGNAME       841
RIVERBASIN    841
OCEANNUM      841
OCEAN         841
geometry      841
dtype: int64

In [32]:
stations_meta['lat_lon_pp'] = gpd.points_from_xy(y =stations_meta['lat_pp'], x =  stations_meta['long_pp'])
pp_points = stations_meta.set_geometry('lat_lon_pp', crs = "EPSG:4326")

In [33]:
stations_meta['lat_lon'] = gpd.points_from_xy(y =stations_meta['lat_org'], x =  stations_meta['long_org'])
org_points = stations_meta.set_geometry('lat_lon', crs = "EPSG:4326")

In [60]:
# m = org_points.explore(style_kwds=dict(color='red'),tooltip=['grdc_no','station','river','quality'])
# pp_points.explore(m=m,style_kwds=dict(color='blue'),tooltip=['grdc_no','station','river','quality'])
# m

In [67]:
def write_topo_features(stations_meta, output_path):
    if isinstance(stations_meta, (Path, str)):
        stations_meta = gpd.read_file(stations_meta)
    topo_df = stations_meta[['grdc_no','lat_pp','long_pp','area_calc']]
    topo_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path),sep=';',index=False)
    print('Saved to',output_path)

In [ ]:
station_fns = [Path(grdc_dir) / f for f in os.listdir(grdc_dir) if 'stationbasins' in f]

output_path = Path(basin_data_dir)/'basin_attributes' / 'topo_attr.txt'
# [write_topo_features(fn,output_path) for fn in station_fns]

In [75]:
full_df = pd.read_csv(output_path,sep=';')


In [77]:
sorted_df = full_df.sort_values(by='grdc_no')
sorted_df.to_csv(output_path, header=True,sep=';',index=False)